In [13]:
CANONICAL_METRICS = {
    "assets": ["Assets"],
    "revenue": ["Revenues", "RevenueFromContractWithCustomerExcludingAssessedTax"],
    "net_income": ["NetIncomeLoss"]
}

def resolve_metric(query: str):
    query = query.lower()
    for _, aliases in CANONICAL_METRICS.items():
        for alias in aliases:
            if alias.lower() in query:
                return alias
    return None

In [14]:
def sort_by_year(docs):
    return sorted(docs, key=lambda d: d.metadata["fiscal_year"])


def compute_yoy_growth(docs):
    docs = sort_by_year(docs)
    results = []

    for i in range(1, len(docs)):
        prev = docs[i-1]
        curr = docs[i]

        yoy = ((curr.metadata["value"] - prev.metadata["value"])
               / prev.metadata["value"]) * 100

        results.append({
            "year": curr.metadata["fiscal_year"],
            "yoy_growth_pct": round(yoy, 2)
        })

    return results

In [15]:
class FinancialQAEngine:
    def __init__(self, retriever):
        self.retriever = retriever

    def answer_trend_question(self, query, company, start_year, end_year):
        metric = resolve_metric(query)

        docs = self.retriever.retrieve(
            query=query,
            company=company,
            metric=metric
        )

        docs = [
            d for d in docs
            if start_year <= d.metadata["fiscal_year"] <= end_year
        ]

        return compute_yoy_growth(docs)